In [1]:
import pandas as pd
import requests
import logging
import time
import json
import math
from elasticsearch import Elasticsearch
import copy
import random
import numpy as np

In [2]:
df_nf_unique = pd.read_csv('notasFiscais/df_nf_unique.csv')
df_nf_unique

/tmp/ipykernel_2837178/1538487853.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_nf_unique = pd.read_csv('notasFiscais/df_nf_unique.csv')


,ID,MODELO,SERIE,NUMERO_NOTA_FISCAL,DATA_EMISSAO,CNPJ_CPF_EMITENTE,RAZAO_SOCIAL_EMITENTE,NOME_FANTASIA_EMITENTE,TIPO_DESTINATARIO,CNPJ_CPF_DESTINATARIO,...,DESCRICAO_PRODUTO,UNIDADE_PRODUTO,UNIDADE_PRODUTO_TRIBUTAVEL,QUANTIDADE_PRODUTO,QUANTIDADE_PRODUTO_TRIBUTAVEL,VALOR_PRODUTO,VALOR_PRODUTO_TRIBUTAVEL,CODIGO_BARRA,CODIGO_BARRA_TRIBUTAVEL,NCM
0,10,55,21,37005,2022-01-01 00:00:00.000,56228356013381,CRBS S/A - CDC Rio Branco,CDC RIO BRANCO,PESSOA_JURIDICA,NaN,...,"SKOL 600ML 6,...",Un,Un,2.0,2.0,5.69,5.69,7891149200405,7891149200405,22030000
1,14,55,21,37005,2022-01-01 00:00:00.000,56228356013381,CRBS S/A - CDC Rio Branco,CDC RIO BRANCO,PESSOA_JURIDICA,NaN,...,"ANTARCTICA PILSEN LATA 350ML SH C/12 NPAL 10,...",Un,Un,12.0,12.0,10.22,10.22,7891991002141,7891991000796,22030000
2,16,55,21,37005,2022-01-01 00:00:00.000,56228356013381,CRBS S/A - CDC Rio Branco,CDC RIO BRANCO,PESSOA_JURIDICA,NaN,...,"PEPSI COLA PET 2L CAIXA C/6 9,...",Un,Un,1.0,1.0,9.61,9.61,7892840800246,7892840800000,22021000
3,53,55,1,2148830,2022-01-01 00:00:00.000,3007331001032,EBAZAR.COM.BR. LTDA,NaN,PESSOA_FISICA,NaN,...,Potes Hermeticos Porta-Mantimentos Tampa Bambu...,UN,UN,1.0,1.0,139.90,139.90,7909569365786,7909569365786,70134290
4,88,55,1,25085,2022-01-01 00:00:00.000,21393527000101,RECOL FARMA LTDA.,RECOL FARMA - 24HORAS,PESSOA_JURIDICA,NaN,...,GRIPINEW C/20 CPR,UN1,UN1,1.0,1.0,8.99,8.99,7896862920029,7896862920029,30045090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2248616,75679419,55,10,12741,2023-07-31 00:00:00.000,5915706000105,COMERCIAL JAPIIM LTDA,COMERCIAL JAPIIM,PESSOA_JURIDICA,NaN,...,ROSQUINHA CHOCOLATE ZERO LOW 150G,UN1,UN1,1.0,1.0,8.07,8.07,7896292070080,7896292070080,19053100
2248617,75679436,55,10,12741,2023-07-31 00:00:00.000,5915706000105,COMERCIAL JAPIIM LTDA,COMERCIAL JAPIIM,PESSOA_JURIDICA,NaN,...,BISC HOT CRACK QUEIJO 150G,UN1,UN1,8.0,8.0,6.26,6.26,7896011105499,7896011105499,19053100
2248618,75679441,55,10,12741,2023-07-31 00:00:00.000,5915706000105,COMERCIAL JAPIIM LTDA,COMERCIAL JAPIIM,PESSOA_JURIDICA,NaN,...,MACARRAO NINHO LARGO PARATI 500G,UN1,UN1,1.0,1.0,5.25,5.25,7896011107028,7896011107028,19021100
2248619,75679480,55,10,12742,2023-07-31 00:00:00.000,5915706000105,COMERCIAL JAPIIM LTDA,COMERCIAL JAPIIM,PESSOA_JURIDICA,NaN,...,GRAFITE ESTAMPADAS,UN1,UN1,1.0,1.0,1.00,1.00,6944504411457,6944504411457,96084000


In [3]:
filtered_df = df_nf_unique[df_nf_unique['DESCRICAO_PRODUTO'].str.split().apply(len) == 5]
filtered_df

,ID,MODELO,SERIE,NUMERO_NOTA_FISCAL,DATA_EMISSAO,CNPJ_CPF_EMITENTE,RAZAO_SOCIAL_EMITENTE,NOME_FANTASIA_EMITENTE,TIPO_DESTINATARIO,CNPJ_CPF_DESTINATARIO,...,DESCRICAO_PRODUTO,UNIDADE_PRODUTO,UNIDADE_PRODUTO_TRIBUTAVEL,QUANTIDADE_PRODUTO,QUANTIDADE_PRODUTO_TRIBUTAVEL,VALOR_PRODUTO,VALOR_PRODUTO_TRIBUTAVEL,CODIGO_BARRA,CODIGO_BARRA_TRIBUTAVEL,NCM
10,915,55,1,28565,2022-01-01 00:00:00.000,14804412000350,SUPERMERCADO PAGUE POUCO LTDA,SUPERMERCADO PAGUE POUCO,PESSOA_JURIDICA,NaN,...,PANO DE CHAO TUDEBOM BRANCO,UN1,UN1,1.0,1.0,8.15,8.15,736532366535,736532366535,63071000
16,1007,55,1,13494,2022-01-01 00:00:00.000,28037020000318,DROGARIA DR FARMA LTDA,NaN,PESSOA_JURIDICA,NaN,...,SERINGA DESC SR 3 ML,UND,UND,38.0,38.0,0.40,0.40,7898259495028,7898259495028,90183119
17,1021,55,1,4709,2022-01-01 00:00:00.000,21393527000799,RECOL FARMA LTDA,RECOL FARMA - GETULIO,PESSOA_JURIDICA,NaN,...,FLOGORAL MENTA C/ 12 PAST,UN1,UN1,1.0,1.0,10.27,10.27,7896658005459,7896658005459,30049099
28,1307,55,7,606,2022-01-01 00:00:00.000,61585865274286,RAIADROGASIL S/A,3631 - RAIADROGASIL S/A,PESSOA_FISICA,NaN,...,"HORA H 1,5MG 1 S",UN,UN,1.0,1.0,18.76,18.76,7898907312646,7898907312646,30066000
29,1358,55,1,261,2022-01-01 00:00:00.000,4521258003371,J CRUZ LTDA,UP FEIJO 33-71,PESSOA_JURIDICA,NaN,...,MUCILON LATA MULTI CEREAIS 400G,UN,UN,6.0,6.0,3.49,3.49,7891000035832,7891000035832,19011030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2248613,75679398,55,10,12741,2023-07-31 00:00:00.000,5915706000105,COMERCIAL JAPIIM LTDA,COMERCIAL JAPIIM,PESSOA_JURIDICA,NaN,...,BISC SALGADINHO PIRAQUE 100G TRA,UN1,UN1,6.0,6.0,3.48,3.48,7896024720115,7896024720115,19053100
2248616,75679419,55,10,12741,2023-07-31 00:00:00.000,5915706000105,COMERCIAL JAPIIM LTDA,COMERCIAL JAPIIM,PESSOA_JURIDICA,NaN,...,ROSQUINHA CHOCOLATE ZERO LOW 150G,UN1,UN1,1.0,1.0,8.07,8.07,7896292070080,7896292070080,19053100
2248617,75679436,55,10,12741,2023-07-31 00:00:00.000,5915706000105,COMERCIAL JAPIIM LTDA,COMERCIAL JAPIIM,PESSOA_JURIDICA,NaN,...,BISC HOT CRACK QUEIJO 150G,UN1,UN1,8.0,8.0,6.26,6.26,7896011105499,7896011105499,19053100
2248618,75679441,55,10,12741,2023-07-31 00:00:00.000,5915706000105,COMERCIAL JAPIIM LTDA,COMERCIAL JAPIIM,PESSOA_JURIDICA,NaN,...,MACARRAO NINHO LARGO PARATI 500G,UN1,UN1,1.0,1.0,5.25,5.25,7896011107028,7896011107028,19021100


In [4]:
most_cod = filtered_df['CODIGO_BARRA'].value_counts()[filtered_df['CODIGO_BARRA'].value_counts() > 25]
most_cod = most_cod.index.tolist()
most_cod


[7893371000372,
 7891033518968,
 7891033496907,
 7891033519033,
 7895800151426,
 7891033517282,
 7896454300031,
 7898505140023,
 7891033492916,
 7896023719899,
 7891033519149,
 7893371000396,
 7891033842162,
 7896454300017,
 7891033500277,
 7891738018954,
 7891033459827,
 7891033500291,
 7898019860905,
 7891738019098,
 751320072559,
 7891033531134,
 7891033497942,
 7898939720068,
 7898505140016,
 7891033531141,
 7891030002033,
 7898031310709,
 7898080640611,
 7896089011357,
 7896440501305,
 7891645028930,
 7891033498017,
 7891033840441]

In [5]:
med_cod = filtered_df['CODIGO_BARRA'].value_counts()[(filtered_df['CODIGO_BARRA'].value_counts() > 15) & (filtered_df['CODIGO_BARRA'].value_counts() < 25)]
med_cod = med_cod.index.tolist()
med_cod


[7891022101003,
 7894900031515,
 7891024194607,
 7899050521244,
 7896001004511,
 7891173023001,
 7891030300207,
 7891738019074,
 7896322701465,
 7896026838245,
 7896040703710,
 7896089012637,
 7897451468632,
 7891032013105,
 7898215151708,
 7894900027013,
 7891040105502,
 7893371000426,
 7898949840053,
 7891033525348,
 7891033531042,
 7896005800362,
 7622300807283,
 7891032013259,
 7893371000358,
 7891097102059,
 7896098905906,
 7897664130036,
 7898933283224,
 7896040703727,
 7891107101621,
 7891504290102,
 7898342050240,
 7891962066073,
 7891000352175,
 7891032012658,
 7894900051513,
 7896026838252,
 7891032015505,
 7898031311324,
 7891008116632,
 7891114008869,
 7896433800378,
 7891132019281,
 7899674017116,
 7896902210646,
 7896023788796,
 7891222002698,
 7897801300308,
 7898010930287,
 7896019602006,
 7896213006235,
 7891033518975,
 7896408440561,
 7896603803796,
 7891738018930,
 7896213005771,
 7891528038810,
 7898505140207,
 7898505140221,
 7891032012504,
 7896026800754,
 7896440

In [6]:
min_cod = filtered_df['CODIGO_BARRA'].value_counts()[(filtered_df['CODIGO_BARRA'].value_counts() > 5) & (filtered_df['CODIGO_BARRA'].value_counts() < 15)]
min_cod = min_cod.index.tolist()
min_cod


[7891114016185,
 7898371002586,
 7897664130319,
 7891738019012,
 7896105177050,
 7896110061221,
 7897801300490,
 7898371000032,
 7622300989316,
 7891249061777,
 7897664140219,
 7891222100028,
 7891000098950,
 7891033530694,
 7896380103553,
 7896024760647,
 7898247780075,
 7896003738629,
 7896000705983,
 7896098904688,
 7897801301626,
 7894900030013,
 7891024037362,
 7896340285671,
 70330129627,
 7899674017109,
 7891035024993,
 7896051115038,
 7899885116998,
 7891435046403,
 611269991000,
 7896380131075,
 7896040706025,
 7896077077167,
 7896040704618,
 7898939720082,
 7891000070673,
 7896408440592,
 7896380105342,
 7896098900222,
 7891024035191,
 7896098900253,
 7891991001342,
 7898068430326,
 7896009301049,
 7898505140030,
 7891150037465,
 7896322701588,
 7891024037690,
 7622300807399,
 7896183910013,
 7891323075195,
 7896733300813,
 7896005279489,
 7896454300079,
 7898026970802,
 7622300990701,
 7622210596413,
 7896098900239,
 7891024183007,
 7891000252604,
 7893590416268,
 7891000035

In [7]:
base_cod = filtered_df['CODIGO_BARRA'].value_counts()[filtered_df['CODIGO_BARRA'].value_counts() < 5]
base_cod = base_cod.index.tolist()
base_cod


[7891317472375,
 7896007551330,
 7892300030688,
 7898133019241,
 7895296118088,
 7899710010415,
 7908134200415,
 7898006194273,
 7896018700536,
 78931251,
 7896009301162,
 7500435132527,
 7891962070124,
 7896181914174,
 7898079000426,
 7891164018399,
 7891097014529,
 7896658011870,
 7895518029321,
 7892300010505,
 7894650013380,
 7898049719426,
 7893590691023,
 7897451414158,
 7898937380653,
 7891055340226,
 7896676422276,
 7891024135020,
 7896012878842,
 7891022860740,
 7891095023318,
 7898205920239,
 7896422517034,
 7898493790378,
 7896183901806,
 7898189491268,
 7899095244085,
 7894916508520,
 7898297960083,
 7891000259351,
 7898215152439,
 7891024184271,
 7896360409064,
 7891097101748,
 7896079587329,
 7896226313085,
 7896014679546,
 7896380191185,
 7891317003180,
 7892840820381,
 7896018800649,
 7896331701234,
 7896408442015,
 7896451900029,
 7896292341098,
 7500435160056,
 7896658039256,
 7896044952794,
 7896725732097,
 7896714215037,
 7897664171473,
 7898465490473,
 789991690034

In [8]:
df_most_cod = filtered_df[filtered_df['CODIGO_BARRA'].isin(most_cod)]
df_most_cod

,ID,MODELO,SERIE,NUMERO_NOTA_FISCAL,DATA_EMISSAO,CNPJ_CPF_EMITENTE,RAZAO_SOCIAL_EMITENTE,NOME_FANTASIA_EMITENTE,TIPO_DESTINATARIO,CNPJ_CPF_DESTINATARIO,...,DESCRICAO_PRODUTO,UNIDADE_PRODUTO,UNIDADE_PRODUTO_TRIBUTAVEL,QUANTIDADE_PRODUTO,QUANTIDADE_PRODUTO_TRIBUTAVEL,VALOR_PRODUTO,VALOR_PRODUTO_TRIBUTAVEL,CODIGO_BARRA,CODIGO_BARRA_TRIBUTAVEL,NCM
1551,37267,55,1,29748,2022-01-03 00:00:00.000,18998273000132,PRIMUS E PRIMUS LTDA,PRIMUS,PESSOA_JURIDICA,NaN,...,PREGO GERDAU C-C 19X39 (3-1-2X9),KG,KG,1.0,1.0,17.00,17.00,7891738019098,7891738019098,73170090
1925,42698,55,1,5072,2022-01-03 00:00:00.000,8803523000188,EWELLYN S SA,EWELLYN,PESSOA_JURIDICA,NaN,...,COPO DESC TOTALPLAST BRANC 180ML,UN,UN,25.0,25.0,5.89,5.89,7898505140023,7898505140023,39241000
2412,50848,55,1,18447,2022-01-03 00:00:00.000,2745979000233,COMERCIAL SIMAO EIRELI EPP FILIAL BOSQUE,COMERCIAL SIMAO BOSQUE,PESSOA_JURIDICA,NaN,...,COPO 180ML BRAN TOTALPLAS 100UN,PC,PC,1.0,1.0,4.23,4.23,7898505140023,7898505140023,39241000
2797,56806,55,1,13383,2022-01-03 00:00:00.000,484273000159,GADELHA IMP E EXP LTDA,COMERCIAL JB,PESSOA_JURIDICA,NaN,...,COPO DESCARTAVEL 180ML TOTAL PLAST,UN,UN,13.0,13.0,5.99,5.99,7898505140023,7898505140023,39241000
2806,56935,55,1,8418,2022-01-03 00:00:00.000,30263026000274,J P IMPORTACAO E EXPORTACAO EIRELI ME,J P MATERIAL DE CONSTRUCAO,PESSOA_FISICA,NaN,...,PREGO 17X27 GERDAU/BELGO PCT 1KG,UN,UN,1.0,1.0,29.42,29.42,7891738018954,7891738018954,73170090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2226864,75089457,55,1,424629,2023-07-27 00:00:00.000,15191890000133,L L B R DISTRIBUIDORA LTDA,NaN,PESSOA_FISICA,NaN,...,ESTJ ARBO PAIS/23 (Desc. 1.73),UN,UN,1.0,1.0,155.46,155.46,7891033531141,7891033531141,33072010
2228842,75155368,55,1,71945,2023-07-28 00:00:00.000,15191890000214,L L B R DISTRIBUIDORA LTDA,NaN,PESSOA_FISICA,NaN,...,ESTJ QUASAR PAIS/23 (Desc. 2.54),UN,UN,1.0,1.0,123.16,123.16,7891033531134,7891033531134,33072010
2242444,75514145,55,1,72088,2023-07-31 00:00:00.000,15191890000214,L L B R DISTRIBUIDORA LTDA,NaN,PESSOA_FISICA,NaN,...,ESTJ MALBEC REG/22 (Desc. 0.70),UN,UN,1.0,1.0,135.92,135.92,7891033492916,7891033492916,34013000
2242927,75533554,55,2,61907,2023-07-31 00:00:00.000,3869526000281,JRD INDUSTRIA FARMACEUTICA LTDA,JRD UNIPHAR - FILIAL DAIA,PESSOA_JURIDICA,NaN,...,PASTA D´AGUA 100G LT:003667 V:07/25,UN,UN,144.0,144.0,4.07,4.07,7898031310709,7898031310709,30039099


In [9]:
df_med_cod = filtered_df[filtered_df['CODIGO_BARRA'].isin(med_cod)]
df_med_cod

,ID,MODELO,SERIE,NUMERO_NOTA_FISCAL,DATA_EMISSAO,CNPJ_CPF_EMITENTE,RAZAO_SOCIAL_EMITENTE,NOME_FANTASIA_EMITENTE,TIPO_DESTINATARIO,CNPJ_CPF_DESTINATARIO,...,DESCRICAO_PRODUTO,UNIDADE_PRODUTO,UNIDADE_PRODUTO_TRIBUTAVEL,QUANTIDADE_PRODUTO,QUANTIDADE_PRODUTO_TRIBUTAVEL,VALOR_PRODUTO,VALOR_PRODUTO_TRIBUTAVEL,CODIGO_BARRA,CODIGO_BARRA_TRIBUTAVEL,NCM
589,18372,55,1,238920,2022-01-02 00:00:00.000,84308980001407,A.C.D.A. IMPORTACAO E EXPORTACAO LTDA,ARAUJOMIX,PESSOA_JURIDICA,NaN,...,REF COCA COLA PET 2L,UN1,UN1,1.0,1.0,6.98,6.98,7894900027013,7894900027013,22021000
596,18454,55,1,94765,2022-01-02 00:00:00.000,84308980000508,A.C.D.A. IMPORTACAO E EXPORTACAO LTDA,ARASUPER AVIARIO,PESSOA_JURIDICA,NaN,...,PAPEL TOALHA STYLUS 50F 2UN,UN1,UN1,1.0,1.0,4.89,4.89,7896026800754,7896026800754,48189090
624,18812,55,1,238931,2022-01-02 00:00:00.000,84308980001407,A.C.D.A. IMPORTACAO E EXPORTACAO LTDA,ARAUJOMIX,PESSOA_JURIDICA,NaN,...,VINAGRE VIRROSAS ALCOOL 750ML COLORIDO,UN1,UN1,1.0,1.0,1.68,1.68,7897006310027,7897006310027,22090000
644,19111,55,1,238915,2022-01-02 00:00:00.000,84308980001407,A.C.D.A. IMPORTACAO E EXPORTACAO LTDA,ARAUJOMIX,PESSOA_JURIDICA,NaN,...,BISC DOCE VITARELLA DELICITA 414G,UN1,UN1,2.0,2.0,4.99,4.99,7896213005771,7896213005771,19053100
658,19437,55,1,44397,2022-01-02 00:00:00.000,84308980000699,A.C.D.A. IMPORTACAO E EXPORTACAO LTDA,ARASUPER FLORESTA,PESSOA_JURIDICA,NaN,...,REF FANTA PET 2L LARANJA,UN1,UN1,1.0,1.0,7.69,7.69,7894900031515,7894900031515,22021000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2242928,75533555,55,2,61907,2023-07-31 00:00:00.000,3869526000281,JRD INDUSTRIA FARMACEUTICA LTDA,JRD UNIPHAR - FILIAL DAIA,PESSOA_JURIDICA,NaN,...,SAL AMARGO 30G LT:003542 V:06/25,UN,UN,240.0,240.0,1.48,1.48,7898031311324,7898031311324,30039099
2243412,75548140,55,1,61,2023-07-31 00:00:00.000,16516276000167,J. G. COSTA LTDA,JORGE MERCANTIL,PESSOA_JURIDICA,NaN,...,GAROTO CAIXA BOMBONS SORTIDOS 250G,UN,UN,5.0,5.0,10.00,10.00,7891008116632,7891008116632,18069000
2243505,75548906,55,1,703,2023-07-31 00:00:00.000,37085399000160,ATACADAO PARANA LTDA,ATACADO PARANA,PESSOA_JURIDICA,NaN,...,LIMPADOR PERF CASA PERFUME AGRADABL,UN1,UN1,4.0,4.0,7.27,7.27,7896040703727,7896040703727,34025000
2247383,75629034,55,1,3177,2023-07-31 00:00:00.000,34709394000173,VALTEMIR BRILHANTE DE OLIVEIRA,COMERCIAL BRILHANTE,PESSOA_JURIDICA,NaN,...,COLHER DESC PRAFESTA BCA 50UN,UN,UN,2.0,2.0,5.00,5.00,7896343070243,7896343070243,39241000


In [10]:
df_min_cod = filtered_df[filtered_df['CODIGO_BARRA'].isin(min_cod)]
df_min_cod

,ID,MODELO,SERIE,NUMERO_NOTA_FISCAL,DATA_EMISSAO,CNPJ_CPF_EMITENTE,RAZAO_SOCIAL_EMITENTE,NOME_FANTASIA_EMITENTE,TIPO_DESTINATARIO,CNPJ_CPF_DESTINATARIO,...,DESCRICAO_PRODUTO,UNIDADE_PRODUTO,UNIDADE_PRODUTO_TRIBUTAVEL,QUANTIDADE_PRODUTO,QUANTIDADE_PRODUTO_TRIBUTAVEL,VALOR_PRODUTO,VALOR_PRODUTO_TRIBUTAVEL,CODIGO_BARRA,CODIGO_BARRA_TRIBUTAVEL,NCM
28,1307,55,7,606,2022-01-01 00:00:00.000,61585865274286,RAIADROGASIL S/A,3631 - RAIADROGASIL S/A,PESSOA_FISICA,NaN,...,"HORA H 1,5MG 1 S",UN,UN,1.0,1.0,18.76,18.76,7898907312646,7898907312646,30066000
29,1358,55,1,261,2022-01-01 00:00:00.000,4521258003371,J CRUZ LTDA,UP FEIJO 33-71,PESSOA_JURIDICA,NaN,...,MUCILON LATA MULTI CEREAIS 400G,UN,UN,6.0,6.0,3.49,3.49,7891000035832,7891000035832,19011030
551,17891,55,1,238910,2022-01-02 00:00:00.000,84308980001407,A.C.D.A. IMPORTACAO E EXPORTACAO LTDA,ARAUJOMIX,PESSOA_FISICA,NaN,...,FLOCOS MILHO SINHA 500G FLOCAO,UN1,UN1,2.0,2.0,1.99,1.99,7892300000933,7892300000933,11041900
552,17893,55,1,238910,2022-01-02 00:00:00.000,84308980001407,A.C.D.A. IMPORTACAO E EXPORTACAO LTDA,ARAUJOMIX,PESSOA_FISICA,NaN,...,ESPONJA ESFREBOM L4P3UN ANTI BACTERIAS,UN1,UN1,1.0,1.0,5.65,5.65,7896001045446,7896001045446,68053090
568,18087,55,1,238913,2022-01-02 00:00:00.000,84308980001407,A.C.D.A. IMPORTACAO E EXPORTACAO LTDA,ARAUJOMIX,PESSOA_JURIDICA,NaN,...,ACHOC PO NESCAU SACH 730G,UN1,UN1,1.0,1.0,8.99,8.99,7891000338162,7891000338162,18069000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2248564,75678792,55,1,30309,2023-07-31 00:00:00.000,9002021000110,R J CARDINAL LTDA,SUPER ECONOMICO,PESSOA_JURIDICA,NaN,...,BISC WAFER PIRAQUE MORANGO 100GR,UN1,UN1,15.0,15.0,2.35,2.35,7896024760654,7896024760654,19053200
2248585,75679124,55,10,12741,2023-07-31 00:00:00.000,5915706000105,COMERCIAL JAPIIM LTDA,COMERCIAL JAPIIM,PESSOA_JURIDICA,NaN,...,EXTRATO DE TOMATE ELEFANTE 190G,UN1,UN1,7.0,7.0,2.80,2.80,7896036095645,7896036095645,20029000
2248603,75679339,55,10,12741,2023-07-31 00:00:00.000,5915706000105,COMERCIAL JAPIIM LTDA,COMERCIAL JAPIIM,PESSOA_JURIDICA,NaN,...,AZEITONAS ZAELI SACHE RECHEADA 150,UN1,UN1,5.0,5.0,6.88,6.88,7896183909604,7896183909604,20057000
2248610,75679379,55,10,12741,2023-07-31 00:00:00.000,5915706000105,COMERCIAL JAPIIM LTDA,COMERCIAL JAPIIM,PESSOA_JURIDICA,NaN,...,MUCILON SNACK TRADICIONAL 35G BR,UN1,UN1,1.0,1.0,3.13,3.13,7891000320242,7891000320242,19059020


In [11]:
base_cod = filtered_df[filtered_df['CODIGO_BARRA'].isin(base_cod)]
base_cod


,ID,MODELO,SERIE,NUMERO_NOTA_FISCAL,DATA_EMISSAO,CNPJ_CPF_EMITENTE,RAZAO_SOCIAL_EMITENTE,NOME_FANTASIA_EMITENTE,TIPO_DESTINATARIO,CNPJ_CPF_DESTINATARIO,...,DESCRICAO_PRODUTO,UNIDADE_PRODUTO,UNIDADE_PRODUTO_TRIBUTAVEL,QUANTIDADE_PRODUTO,QUANTIDADE_PRODUTO_TRIBUTAVEL,VALOR_PRODUTO,VALOR_PRODUTO_TRIBUTAVEL,CODIGO_BARRA,CODIGO_BARRA_TRIBUTAVEL,NCM
10,915,55,1,28565,2022-01-01 00:00:00.000,14804412000350,SUPERMERCADO PAGUE POUCO LTDA,SUPERMERCADO PAGUE POUCO,PESSOA_JURIDICA,NaN,...,PANO DE CHAO TUDEBOM BRANCO,UN1,UN1,1.0,1.0,8.15,8.15,736532366535,736532366535,63071000
16,1007,55,1,13494,2022-01-01 00:00:00.000,28037020000318,DROGARIA DR FARMA LTDA,NaN,PESSOA_JURIDICA,NaN,...,SERINGA DESC SR 3 ML,UND,UND,38.0,38.0,0.40,0.40,7898259495028,7898259495028,90183119
32,1460,55,1,5547,2022-01-01 00:00:00.000,4521258002057,J CRUZ LTDA - UP CEARA,ULTRA POPULAR CEARA 20-57,PESSOA_FISICA,NaN,...,RESFENOL 10CAPS - Lab./Fabr. HERTZ,UN,UN,1.0,1.0,11.31,11.31,7896331705720,7896331705720,30039055
60,1764,55,1,158260744,2022-01-01 00:00:00.000,56991441000823,AVON COSMETICOS LTDA,NaN,PESSOA_FISICA,NaN,...,COL DES FULL SPEED NITRO,UN,UN,1.0,1.0,28.68,28.68,7909189090952,7909189090952,33072010
65,1778,55,1,158260743,2022-01-01 00:00:00.000,56991441000823,AVON COSMETICOS LTDA,NaN,PESSOA_FISICA,NaN,...,ERVA DOCE DESODORANTE AEROSOL ANTITRANSPIRANT,UN,UN,1.0,1.0,11.00,11.00,7909189169740,7909189169740,33072010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2248605,75679342,55,10,12741,2023-07-31 00:00:00.000,5915706000105,COMERCIAL JAPIIM LTDA,COMERCIAL JAPIIM,PESSOA_JURIDICA,NaN,...,AMENDOIM TIPO JAPONES 200G DORI,UN1,UN1,7.0,7.0,5.66,5.66,7896058502107,7896058502107,20081100
2248616,75679419,55,10,12741,2023-07-31 00:00:00.000,5915706000105,COMERCIAL JAPIIM LTDA,COMERCIAL JAPIIM,PESSOA_JURIDICA,NaN,...,ROSQUINHA CHOCOLATE ZERO LOW 150G,UN1,UN1,1.0,1.0,8.07,8.07,7896292070080,7896292070080,19053100
2248617,75679436,55,10,12741,2023-07-31 00:00:00.000,5915706000105,COMERCIAL JAPIIM LTDA,COMERCIAL JAPIIM,PESSOA_JURIDICA,NaN,...,BISC HOT CRACK QUEIJO 150G,UN1,UN1,8.0,8.0,6.26,6.26,7896011105499,7896011105499,19053100
2248618,75679441,55,10,12741,2023-07-31 00:00:00.000,5915706000105,COMERCIAL JAPIIM LTDA,COMERCIAL JAPIIM,PESSOA_JURIDICA,NaN,...,MACARRAO NINHO LARGO PARATI 500G,UN1,UN1,1.0,1.0,5.25,5.25,7896011107028,7896011107028,19021100


In [12]:
most_random_desc = df_most_cod.sample(10)
most_random_desc
most_queries = most_random_desc['DESCRICAO_PRODUTO'].to_list()
most_queries

['ALCOOL SANTA CRUZ 96 12X1L',
 'LIMP ALUM POLITRIZ TRADIC 500ML',
 'LIMA ENXADA 8 - K&F',
 'ESTJ LIZ MAES/23 (Desc. 2.51)',
 'ESTJ LINDA MAES/23 (Desc. 1.24)',
 'MIUDO DE BOVINO CONG (RABO).....15CX',
 'ESTJ UOMINI NAT/22 (Desc. 2.67)',
 'LEITE ITALAC INTEGRAL 1 LT',
 'ESTJ LIZ MAES/23 (Desc. 5.59)',
 'ESTJ MALBEC NAT/22 (Desc. 1.27)']

In [13]:
med_random_desc = df_med_cod.sample(10)
med_random_desc
med_queries = med_random_desc['DESCRICAO_PRODUTO'].to_list()
med_queries

['ESTJ ZAAD NAT/22 (Desc. 14.45)',
 'REFRIGERANTE DE GUARANA BARE 2LTS',
 'FITA ADESIVA TRANSP 48MMX40MTS QUALITAPE',
 'ESPONJA ACO 60G BOM BRIL',
 'VINAGRE DE ALCOOL COLORIDO 750ML',
 'VASSOURA IMPERADOR N°4 C/ CABO',
 'TAMPICO 2 LT SABOR LARANJA',
 'SARDINHA COM OLEO NAUTIQUE 125G',
 'LIMP ALUM POLITRIZ LIMAO 12X500ML',
 'JOELHO 90 SOLD 25MM KRONA']

In [14]:
min_random_desc = df_min_cod.sample(10)
min_random_desc
min_queries = min_random_desc['DESCRICAO_PRODUTO'].to_list()
min_queries

['CLORIDRATO DE DILTIAZEM 60MG 50CP',
 'PREGO CCAB 15x15-1.1/4x13 POL CX',
 'ALCOOL 70o/o SANTA CRUZ 500ML',
 'PAPEL HIG.MAX PURE 30M L12P11',
 'BUDESONIDA GEN SPRAY 32MCG 120DOSE',
 'LTE COND PIRACANJUBA TP 395G',
 'IBUPROTRAT SUSP 20MG/ML 100ML INF+SER',
 'REFRESCO TANG LARANJA 25G (150)',
 'CR INTENSIVO NIVEA SOFT 98G',
 'VEDA CALHA CINZA PC 280G']

In [15]:
base_random_desc = base_cod.sample(10)
base_random_desc
base_queries = base_random_desc['DESCRICAO_PRODUTO'].to_list()
base_queries

['SQUEEZE PLAST ALEGRIA 550ML CB1042',
 'CARRO DE PASSEIO LOCOMOTIVA CALESITA',
 'Sob Batavo Flan 400g Caramelo',
 'MOLHO SALADA WILSON 250ML CAESAR',
 'CAMISA ATR MC 12010 G',
 'PRESTIGIO CookieChoc GotasChoc 52x60g BR',
 'BANDEJA HIGIENICA S/BORDA 41,5X31,5X9CM 743',
 'PASTA IPAD SONHOS SEM REALIZAM',
 'BATATA PALHA UAU CHIPS 70G',
 'Tela Decorativa Paz - Medio']

In [16]:
repeticao = 10

def create_queries(resp, queries, type):
    for j in range(repeticao):
        for query in queries:
            splited = query.split()
            for i in range(len(splited)):
                    resp.append({'query': ' '.join(splited[0: i+1]), 'type': type, 'querySize': i+1, 'repetition': j+1})

resp = []
create_queries(resp, most_queries, 'max')
create_queries(resp, med_queries, 'med1')
create_queries(resp, min_queries, 'med2')
create_queries(resp, base_queries, 'min')


random.shuffle(resp)
time_queries = []
for q in resp:
    q_allmini = copy.deepcopy(q)
    q_allmini['abordagem'] = 'all_mini'
    time_queries.append(q_allmini)
    
    q_bv2 = copy.deepcopy(q)
    q_bv2['abordagem'] = 'q_bv2'
    time_queries.append(q_bv2)

    q_mpnet = copy.deepcopy(q)
    q_mpnet['abordagem'] = 'mpnet'
    time_queries.append(q_mpnet)
    
    q_labase = copy.deepcopy(q)
    q_labase['abordagem'] = 'labase'
    time_queries.append(q_labase)
df_test_time = pd.DataFrame(time_queries)
df_test_time

,query,type,querySize,repetition,abordagem
0,MOLHO SALADA,min,2,8,all_mini
1,MOLHO SALADA,min,2,8,q_bv2
2,MOLHO SALADA,min,2,8,mpnet
3,MOLHO SALADA,min,2,8,labase
4,BUDESONIDA GEN,med2,2,7,all_mini
...,...,...,...,...,...
7995,LIMA ENXADA 8,max,3,2,labase
7996,LIMP ALUM,med1,2,3,all_mini
7997,LIMP ALUM,med1,2,3,q_bv2
7998,LIMP ALUM,med1,2,3,mpnet


In [17]:
df_test_time[['querySize']].value_counts()

querySize
1            1600
2            1600
3            1600
4            1600
5            1600
Name: count, dtype: int64

In [18]:
df_test_time.to_csv("ruidosCsvEmbeddings/sample_test_NF_tempo_embeddings.csv")

In [4]:
df_test_time = pd.read_csv("ruidosCsvEmbeddings/sample_test_NF_tempo_embeddings.csv")
df_test_time

,Unnamed: 0,query,type,querySize,repetition,abordagem,time
0,0,MOLHO SALADA,min,2,8,all_mini,NaN
1,1,MOLHO SALADA,min,2,8,q_bv2,NaN
2,2,MOLHO SALADA,min,2,8,mpnet,NaN
3,3,MOLHO SALADA,min,2,8,labase,NaN
4,4,BUDESONIDA GEN,med2,2,7,all_mini,NaN
...,...,...,...,...,...,...,...
7995,7995,LIMA ENXADA 8,max,3,2,labase,NaN
7996,7996,LIMP ALUM,med1,2,3,all_mini,NaN
7997,7997,LIMP ALUM,med1,2,3,q_bv2,NaN
7998,7998,LIMP ALUM,med1,2,3,mpnet,NaN


In [3]:
# es = Elasticsearch(
#     "http://150.165.75.163:9200",
#     basic_auth=("elastic", "GnlrL41DQapI3"),
#     request_timeout=999999999
# )

# es.info()
es = Elasticsearch(
    "http://150.165.75.163:9202",
    basic_auth=("elastic", "teste123"),
    request_timeout=999999999
)

es.info()


ObjectApiResponse({'name': 'es01', 'cluster_name': 'es-docker-cluster', 'cluster_uuid': '8pFe7X9oRge9WST4LcJmzw', 'version': {'number': '8.6.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '2d58d0f136141f03239816a4e360a8d17b6d8f29', 'build_date': '2023-02-13T09:35:20.314882762Z', 'build_snapshot': False, 'lucene_version': '9.4.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [20]:
indices = es.cat.indices(h='index', format='json')
nomes_indices = [indice['index'] for indice in indices]
nomes_indices

['matlex', 'index_lex_nf', 'index_s_nf', 'mat_semaanc']

In [21]:
destino_consulta = 'index_lex_nf'
match_query = 'fone wireless'

query = {
        "query": {
            "match": {
                "descricao_produto": match_query 
            }
        }
    }

response = es.search(index=destino_consulta, body=query)
for hit in response['hits']['hits']:
    print(hit['_source']['descricao_produto']) 


WIRELESS HEADSET FONE PR
FONE DE OUVIDO WIRELESS COM LED - PRATA
FONE DE OUVIDO WIRELESS COM LED - VERMELHO
Fone True Wireless Earbuds - Champion
FONE DE OUVIDO WIRELESS HS-391 HREBOS
FONE DE OUV WIRELESS EARPHONE AZUL TMV
Fone De Ouvido Wireless Bluetooth Inpods 12 Branco
FONE DE OUVIDO SEM FIO WIRELESS STEREO HEADPHONES B05 PEINING
Fone Ouvido Awei B925bl Bluetooth Wireless Esporte Earphone
Fone de Ouvido Wireless Bluetooth InPods 12 Azul Marinho - Booglee


In [2]:
from sentence_transformers import SentenceTransformer
modelL6V2 = SentenceTransformer('all-MiniLM-L6-v2')
modelBV2 = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
modelMult = SentenceTransformer('sentence-transformers/quora-distilbert-multilingual')
modelLASE = SentenceTransformer('sentence-transformers/LaBSE')

/home/melqui/miniconda3/envs/jupyter-env/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/melqui/miniconda3/envs/jupyter-env/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/home/melqui/miniconda3/envs/jupyter-env/lib/python3.12/site-packages/sentence_transformers/models/Dense.py:89: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is

In [5]:
destino_consulta = 'index_s_nf'
match_query = modelL6V2.encode('fone wireless')

query_smt= {
    'field': 'all_mini_base_vector',
    'query_vector': match_query.tolist(),
    'k': 10,
    'num_candidates': 50
}


response = es.search(index=destino_consulta, knn=query_smt)

for hit in response['hits']['hits']:
    print(hit['_source']['descricao_produto'])

WIRELESS HEADSET FONE PR
fone
FONE BLUETOOTH TWS AIRPLUS(F80)
FONE BLUETOOTH TWS ET31
FONE BLUETOOTH TWS E10
FONE BLUETOOTH TWS ET31(F70 PLUS)
FONE BLUETOOTH MAKETECH AP-20
FOTOPLIMERIZADOR EMITTER NOW WIRELESS - SCHUSTER
FONE BLUETOOTH TWS X23
Fone True Wireless Earbuds - Champion


In [24]:
from requests.auth import HTTPBasicAuth

index_consulta = 'index_s_nf'
match_query = modelL6V2.encode('fone wireless')
print(index_consulta)
print('----------')

script_query = {
    "query": {
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": "cosineSimilarity(params.query_vector, 'all_mini_base_vector') + 1.0",
                "params": {"query_vector": match_query.tolist()}
            }
        }
    }
    }
start_time = time.perf_counter()

response = requests.get('http://150.165.75.163:9202/' + index_consulta + '/_search', json=query, headers={'Content-type': 'application/json'},  auth=HTTPBasicAuth('elastic', 'teste123'))
time_elapsed = time.perf_counter() - start_time
total_time = time_elapsed
print(response)
print('--------------')
print(requests.codes.ok)
if response.status_code == requests.codes.ok:
    response_data = json.loads(response.content)
    
    hits = response_data['hits']['hits']
    
    modelos = [hit['_source']['descricao_produto'] for hit in hits]
    print(modelos)


index_s_nf
----------
<Response [200]>
--------------
200
['WIRELESS HEADSET FONE PR', 'Fone True Wireless Earbuds - Champion', 'FONE DE OUVIDO WIRELESS COM LED - PRATA', 'FONE DE OUVIDO WIRELESS HS-391 HREBOS', 'FONE DE OUVIDO WIRELESS COM LED - VERMELHO', 'FONE DE OUV WIRELESS EARPHONE AZUL TMV', 'Fone Ouvido Awei B925bl Bluetooth Wireless Esporte Earphone', 'Fone De Ouvido Wireless Bluetooth Inpods 12 Branco', 'Fone de Ouvido Wireless Bluetooth InPods 12 Azul Marinho - Booglee', 'Fone de Ouvido (Headset) Alienware Wireless Gaming - AW920H LUNAR LIGHT']


In [6]:
import gc
import torch
import logging

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
from requests.auth import HTTPBasicAuth
logging.basicConfig(filename= 'ruidosCsvEmbeddings/experiment_temp_embeddings_NF.log',
    filemode='a',
    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
    datefmt='%H:%M:%S',
    level=logging.DEBUG)


def run(file, start=0, half=False):
    total_time = 0
    # processed_queries = 0
    experiment_queries = pd.read_csv(file)
    # experiment_queries['result'] = None

    num_rows = len(experiment_queries)
    mid_point = num_rows // 2

    if half:
        start = mid_point
        end = num_rows
    else:
        end = mid_point

    print("start: ", start)
    print("end: ", end)
    
    for key, value in experiment_queries.iloc[start:end].iterrows():    
        if(key % 800 == 0):
            print('key', key)
        logging.info(f'Iniciando processamento da query ({value["repetition"]}): {value["query"]})')
        try:

            match_query = ''
            query = ''          
            if(value['abordagem']) == 'all_mini':
                    index_consulta = 'index_s_nf'
                    match_query = modelL6V2.encode(value['query'],show_progress_bar = False)
                    
                    query = {
                        "query": {
                            "script_score": {
                                "query": {"match_all": {}},
                                "script": {
                                    "source": "cosineSimilarity(params.query_vector, 'all_mini_base_vector') + 1.0",
                                    "params": {"query_vector": match_query.tolist()}
                                }
                            }
                        }
                    }
            
            elif(value['abordagem']) == 'q_bv2':
                    index_consulta = 'index_s_nf'
                    match_query = modelBV2.encode(value['query'],show_progress_bar = False)
                    
                    query = {
                        "query": {
                            "script_score": {
                                "query": {"match_all": {}},
                                "script": {
                                    "source": "cosineSimilarity(params.query_vector, 'bv2_base_vector') + 1.0",
                                    "params": {"query_vector": match_query.tolist()}
                                }
                            }
                        }
                    }
     
            elif(value['abordagem']) == 'mpnet':
                    index_consulta = 'index_s_nf'
                    match_query = modelMult.encode(value['query'],show_progress_bar = False)
                    
                    query = {
                        "query": {
                            "script_score": {
                                "query": {"match_all": {}},
                                "script": {
                                    "source": "cosineSimilarity(params.query_vector, 'mpnet_base_vector') + 1.0",
                                    "params": {"query_vector": match_query.tolist()}
                                }
                            }
                        }
                    }
            elif(value['abordagem']) == 'labase':
                    index_consulta = 'index_s_nf'
                    match_query = modelLASE.encode(value['query'],show_progress_bar = False)
                    
                    query = {
                        "query": {
                            "script_score": {
                                "query": {"match_all": {}},
                                "script": {
                                    "source": "cosineSimilarity(params.query_vector, 'labase_base_vector') + 1.0",
                                    "params": {"query_vector": match_query.tolist()}
                                }
                            }
                        }
                    }    

            sucess = False
            while(not sucess):
                start_time = time.perf_counter()
                response = requests.get('http://150.165.75.163:9202/' + index_consulta + '/_search', json=query, headers={'Content-type': 'application/json'},  auth=HTTPBasicAuth('elastic', 'teste123'))
                
                time_elapsed = time.perf_counter() - start_time
                total_time += time_elapsed
                
                if response.status_code == requests.codes.ok:
                    sucess = True
                
            # processed_queries += 1
            # experiment_queries['result'][key] = response.content
            experiment_queries['time'][key] = time_elapsed

            # remaining_time = (total_time / processed_queries) * (len(experiment_queries) - processed_queries)

            logging.info(f'A query levou {time_elapsed:.6f}s')
            # logging.info(f'Faltam aproximadamente {int(remaining_time // 3600)}h, {int((remaining_time % 3600) // 60)}m e {math.floor(remaining_time % 60)}s')
        
        except Exception as error:
            logging.error(f'Erro na consulta: {error}')
    experiment_queries.to_csv(file)

In [11]:
file_name = '/mnt/DADOS/melqui/busca-smt/ruidosCsvEmbeddings/sample_test_NF_tempo_embeddings.csv'
# df_test_time['result'] = None
# df_test_time['time'] = None
# df_test_time.to_csv(file_name)


In [9]:
df_test_time

,Unnamed: 0,query,type,querySize,repetition,abordagem,time
0,0,MOLHO SALADA,min,2,8,all_mini,NaN
1,1,MOLHO SALADA,min,2,8,q_bv2,NaN
2,2,MOLHO SALADA,min,2,8,mpnet,NaN
3,3,MOLHO SALADA,min,2,8,labase,NaN
4,4,BUDESONIDA GEN,med2,2,7,all_mini,NaN
...,...,...,...,...,...,...,...
7995,7995,LIMA ENXADA 8,max,3,2,labase,NaN
7996,7996,LIMP ALUM,med1,2,3,all_mini,NaN
7997,7997,LIMP ALUM,med1,2,3,q_bv2,NaN
7998,7998,LIMP ALUM,med1,2,3,mpnet,NaN


In [18]:
run(file_name)

start:  0
end:  4000
key 0
key 800
key 1600
key 2400
key 3200


In [19]:
run(file_name, half=True)

start:  4000
end:  8000
key 4000
key 4800
key 5600
key 6400
key 7200


In [22]:
result = pd.read_csv('ruidosCsvEmbeddings/sample_test_NF_tempo_embeddings.csv')
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0.2  8000 non-null   int64  
 1   Unnamed: 0.1  8000 non-null   int64  
 2   Unnamed: 0    8000 non-null   int64  
 3   query         8000 non-null   object 
 4   type          8000 non-null   object 
 5   querySize     8000 non-null   int64  
 6   repetition    8000 non-null   int64  
 7   abordagem     8000 non-null   object 
 8   time          8000 non-null   float64
dtypes: float64(1), int64(5), object(3)
memory usage: 562.6+ KB


In [23]:
result.describe()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,querySize,repetition,time
count,8000.00000,8000.00000,8000.00000,8000.000000,8000.000000,8000.000000
mean,3999.50000,3999.50000,3999.50000,3.000000,5.500000,0.705939
std,2309.54541,2309.54541,2309.54541,1.414302,2.872461,0.459853
min,0.00000,0.00000,0.00000,1.000000,1.000000,0.154008
25%,1999.75000,1999.75000,1999.75000,2.000000,3.000000,0.328540
50%,3999.50000,3999.50000,3999.50000,3.000000,5.500000,0.448008
75%,5999.25000,5999.25000,5999.25000,4.000000,8.000000,1.251550
max,7999.00000,7999.00000,7999.00000,5.000000,10.000000,3.261491


In [24]:
result.groupby(['abordagem']).agg({'time': ['mean', 'std']})

time          
               mean       std
abordagem                    
all_mini   0.417833  0.243807
labase     0.794269  0.464135
mpnet      0.812032  0.489900
q_bv2      0.799621  0.469485

# Comparação  com léxico
Comparação do embedding de melhor Resultado com a abordagem Léxica

In [25]:
repeticao = 10

def create_queries(resp, queries, type):
    for j in range(repeticao):
        for query in queries:
            splited = query.split()
            for i in range(len(splited)):
                    resp.append({'query': ' '.join(splited[0: i+1]), 'type': type, 'querySize': i+1, 'repetition': j+1})

resp = []
create_queries(resp, most_queries, 'max')
create_queries(resp, med_queries, 'med1')
create_queries(resp, min_queries, 'med2')
create_queries(resp, base_queries, 'min')


random.shuffle(resp)
time_queries = []
for q in resp:
     q_semantic = copy.deepcopy(q)
     q_semantic['abordagem'] = 'semantico'
     time_queries.append(q_semantic)
     q_lexico = copy.deepcopy(q)
     q_lexico['abordagem'] = 'lexico'
     time_queries.append(q_lexico)

df_time_lx_smnt = pd.DataFrame(time_queries)
df_time_lx_smnt

,query,type,querySize,repetition,abordagem
0,ESPONJA ACO 60G BOM BRIL,med1,5,4,semantico
1,ESPONJA ACO 60G BOM BRIL,med1,5,4,lexico
2,Sob Batavo Flan,min,3,10,semantico
3,Sob Batavo Flan,min,3,10,lexico
4,FITA ADESIVA,med1,2,6,semantico
...,...,...,...,...,...
3995,PRESTIGIO CookieChoc GotasChoc 52x60g,min,4,4,lexico
3996,BATATA PALHA UAU CHIPS 70G,min,5,4,semantico
3997,BATATA PALHA UAU CHIPS 70G,min,5,4,lexico
3998,BATATA PALHA UAU CHIPS,min,4,4,semantico


In [26]:
df_time_lx_smnt[['querySize']].value_counts()

querySize
1            800
2            800
3            800
4            800
5            800
Name: count, dtype: int64

In [27]:
df_time_lx_smnt.to_csv("ruidosCsvEmbeddings/sample_test_NF_tempo_lx_smnt.csv")

In [28]:
from requests.auth import HTTPBasicAuth
logging.basicConfig(filename= 'ruidosCsvEmbeddings/experiment_temp_embeddings_NF_lx_smt.log',
    filemode='a',
    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
    datefmt='%H:%M:%S',
    level=logging.DEBUG)

def run_lx_smt(file):
    total_time = 0
    processed_queries = 0
    experiment_queries = pd.read_csv(file)
    # experiment_queries['result'] = None
    for key, value in experiment_queries.iterrows():
        if(key % 800 == 0):
            print('key', key)
        logging.info(f'Iniciando processamento da query ({value["repetition"]}): {value["query"]})')
        try:

            match_query = ''
            query = ''
            if(value['abordagem']) == 'semantico':
                index_consulta = 'index_s_nf'
                match_query = modelL6V2.encode(value['query'],show_progress_bar = False)
                
                query = {
                    "query": {
                        "script_score": {
                            "query": {"match_all": {}},
                            "script": {
                                "source": "cosineSimilarity(params.query_vector, 'all_mini_base_vector') + 1.0",
                                "params": {"query_vector": match_query.tolist()}
                            }
                        }
                    }
                }
            
            else:
                index_consulta = 'index_lex_nf'
                match_query = value['query']
                
                query = {
                    "query": {
                        "match": {
                            "descricao_produto": match_query
                        }
                    }
                }    

            sucess = False
            while(not sucess):
                start_time = time.perf_counter()
                
                response = requests.get('http://150.165.75.163:9202/' + index_consulta + '/_search', json=query, headers={'Content-type': 'application/json'},  auth=HTTPBasicAuth('elastic', 'teste123'))
                time_elapsed = time.perf_counter() - start_time
                total_time += time_elapsed
                
                if response.status_code == requests.codes.ok:
                    sucess = True
            
            
            processed_queries += 1
            # experiment_queries['result'][key] = response.content
            experiment_queries['time'][key] = time_elapsed

            remaining_time = (total_time / processed_queries) * (len(experiment_queries) - processed_queries)

            logging.info(f'A query levou {time_elapsed:.6f}s')
            logging.info(f'Faltam aproximadamente {int(remaining_time // 3600)}h, {int((remaining_time % 3600) // 60)}m e {math.floor(remaining_time % 60)}s')
            gc.collect()
            torch.cuda.empty_cache()
        except Exception as error:
            logging.error(f'Erro na consulta: {error}')
    experiment_queries.to_csv(file)

In [30]:
file_name = '/mnt/DADOS/melqui/busca-smt/ruidosCsvEmbeddings/sample_test_NF_tempo_lx_smnt.csv'
# df_time_lx_smnt['result'] = None
df_time_lx_smnt['time'] = None
df_time_lx_smnt.to_csv(file_name)


In [31]:
df_time_lx_smnt

,query,type,querySize,repetition,abordagem,time
0,ESPONJA ACO 60G BOM BRIL,med1,5,4,semantico,None
1,ESPONJA ACO 60G BOM BRIL,med1,5,4,lexico,None
2,Sob Batavo Flan,min,3,10,semantico,None
3,Sob Batavo Flan,min,3,10,lexico,None
4,FITA ADESIVA,med1,2,6,semantico,None
...,...,...,...,...,...,...
3995,PRESTIGIO CookieChoc GotasChoc 52x60g,min,4,4,lexico,None
3996,BATATA PALHA UAU CHIPS 70G,min,5,4,semantico,None
3997,BATATA PALHA UAU CHIPS 70G,min,5,4,lexico,None
3998,BATATA PALHA UAU CHIPS,min,4,4,semantico,None


In [32]:
run_lx_smt(file_name)

key 0
key 800
key 1600
key 2400
key 3200


In [33]:
result_lx_smn = pd.read_csv('ruidosCsvEmbeddings/sample_test_NF_tempo_lx_smnt.csv')
result_lx_smn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0.1  4000 non-null   int64  
 1   Unnamed: 0    4000 non-null   int64  
 2   query         4000 non-null   object 
 3   type          4000 non-null   object 
 4   querySize     4000 non-null   int64  
 5   repetition    4000 non-null   int64  
 6   abordagem     4000 non-null   object 
 7   time          4000 non-null   float64
dtypes: float64(1), int64(4), object(3)
memory usage: 250.1+ KB


In [35]:
result_lx_smn.describe()

,Unnamed: 0.1,Unnamed: 0,querySize,repetition,time
count,4000.000000,4000.000000,4000.00000,4000.00000,4000.000000
mean,1999.500000,1999.500000,3.00000,5.50000,0.101410
std,1154.844867,1154.844867,1.41439,2.87264,0.099335
min,0.000000,0.000000,1.00000,1.00000,0.001886
25%,999.750000,999.750000,2.00000,3.00000,0.002845
50%,1999.500000,1999.500000,3.00000,5.50000,0.088896
75%,2999.250000,2999.250000,4.00000,8.00000,0.200222
max,3999.000000,3999.000000,5.00000,10.00000,0.683178


In [36]:
result_lx_smn.groupby(['abordagem']).agg({'time': ['mean', 'std']})

time          
               mean       std
abordagem                    
lexico     0.003373  0.001754
semantico  0.199448  0.022456